<h1> DATA RPOCESSING: TORONTO NEIGHBOURHOODS </h1>

<h3>Part one: obtain and cleand data</h3>

In this section data should be imported from Wikipedia and cleaned to satisfy these criteria:
<ul><li>
    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
<li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
    
<li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
    
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</li>
</ul>

In [38]:
#pip install geocoder

In [39]:
#pip install folium

In [40]:
import pandas as pd
import numpy as np
import geocoder

Import data and exclude rows where Borough is not assigned

In [51]:
x = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

x = x[x.Borough != "Not assigned"]
x.reset_index(inplace = True)
x.head()
x.shape

(210, 4)

For neigbourhoods without a name (not assigned) name of Bourough should be assigned:

In [52]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for i in range(0, x.shape[0]):
    if x.Neighbourhood[i]=="Not assigned":
        x.Neighbourhood[i]=x.Borough[i]

In order to merge all Neigbourhoods with a comma for the same postal codes, let's firstly sort data by Postal Code and then iterate by rows. If postal code of row is same with previous row value then neigbourhood should be concatenated with previous ones. <br> Also let's introduce new column Keep = 0 and if current postal code is not equal to next one, then keep = 1. Finally, keep only rows where corresponding column equal to 1 and final row. Then drop keep columns as it's not needed column.

In [53]:
#sort df by postal code
x.initial_shape = range(0, x.shape[0])
x.sort_values(by = "Postcode")

x['Keep'] = 0



x.shape
#loop trough dataframe to append Neighborhoods for the same Postal Codes
#Assign 1 in column keep for rows which hold the last row for each Postal Code, as it contains all the neighbourhoods
for i in range(1, x.shape[0]):
    if x.Postcode[i]==x.Postcode[i-1]:
        x.Neighbourhood[i] = x.Neighbourhood[i-1] + ", " + x.Neighbourhood[i] 
        
for i in range(0, x.shape[0]-1):
    if x.Postcode[i]!=x.Postcode[i+1]:
        x.Keep[i]=1
#As loop always look at i+1 the last row was uncovered. Last row always should be kept as it always last row of a Postal COde.        
x.Keep[x.shape[0]-1]=1   

#keep only rows with keep == 1, so there would be rows with unique postal codes and all their neighbourhoods.

x = x[x.Keep == 1]
x.reset_index(inplace = True)
x.drop(['Keep', 'index', 'level_0'], axis = 1, inplace = True)    
    
#Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
x.head()


C:\Users\Amalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\Users\Amalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Amalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Amalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [54]:
x.shape

(103, 3)

So the final dataframe for part one consist of 103 rows

<h3>PART TWO: OBTAIN FINAL DATASET WITH COORDINATED</h3>

<p>I honestly tried to obtain coordinates using geocoder, but request is denied. After googling (what an irony) the problem, it could be because some additional parameters are not defiend. Nevertheless, this package can sometimes returns NA, so let's use prepared file.</p>

In [55]:
g = geocoder.google('Mountain View, CA')
g

<[REQUEST_DENIED] Google - Geocode [empty]>

So here is the coordinated file:

In [56]:
coords_file = pd.read_csv("https://cocl.us/Geospatial_data")
coords_file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


So here each row stand for Postal Code and coordinates. We can do left join by postal code:

In [57]:
x = pd.merge(x, coords_file, left_on = 'Postcode', right_on = 'Postal Code', how = 'left')
x.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


<h3>PART THREE: CLUSTERIZATOIN </h3>

Let's use functions prepared in the course Lab to obtain all the venues for given neigbourhoods and obtain top-10 categories. Then investigate given results to build clusters.

In [58]:
import requests # library to handle requests

In [59]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

If you would like to run the code yourself, insert your Foursquare API credentials:

In [62]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [63]:
# type your answer here

toronto_venues = getNearbyVenues(names=x['Neighbourhood'],
                                   latitudes=x['Latitude'],
                                   longitudes=x['Longitude']
                                  )

So we got now final dataset:

In [64]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [65]:
toronto_venues.shape

(2241, 7)

We can see that there are more than 2000 venues. To what category do they belong?

In [66]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop                      182
Café                              99
Restaurant                        66
Park                              54
Pizza Place                       52
Bakery                            52
Italian Restaurant                49
Sandwich Place                    43
Bar                               42
Hotel                             42
Clothing Store                    37
Japanese Restaurant               35
Gym                               33
Fast Food Restaurant              31
American Restaurant               30
Sushi Restaurant                  28
Burger Joint                      26
Pub                               25
Breakfast Spot                    24
Seafood Restaurant                24
Gastropub                         23
Thai Restaurant                   23
Grocery Store                     22
Steakhouse                        22
Diner                             21
Chinese Restaurant                21
Pharmacy                          20
B

Most common places are caffes and coffe shops, as well as another restaurants, bars, i.e. places to have meal and rest. Let's take a look of all unique places types:

In [67]:
toronto_venues['Venue Category'].unique()

array(['Park', 'Food & Drink Shop', 'Hockey Arena', 'Coffee Shop',
       'Portuguese Restaurant', 'French Restaurant', 'Bakery',
       'Gym / Fitness Center', 'Spa', 'Breakfast Spot', 'Restaurant',
       'Historic Site', 'Pub', 'Chocolate Shop', 'Farmers Market',
       'Dessert Shop', 'Performing Arts Venue', 'Mexican Restaurant',
       'Café', 'Greek Restaurant', 'Theater', 'Italian Restaurant',
       'Event Space', 'Art Gallery', 'Ice Cream Shop', 'Shoe Store',
       'Cosmetics Shop', 'Asian Restaurant', 'Brewery',
       'Electronics Store', 'Bank', 'Beer Store', 'Hotel',
       'Health Food Store', 'Antique Shop', 'Boutique',
       'Furniture / Home Store', 'Vietnamese Restaurant',
       'Clothing Store', 'Accessories Store', 'Fraternity House',
       'Carpet Store', 'Miscellaneous Shop', 'Gym', 'Sushi Restaurant',
       'Creperie', 'Burrito Place', 'Yoga Studio', 'Persian Restaurant',
       'Hobby Shop', 'Arts & Crafts Store', 'Diner', 'Beer Bar',
       'Wings Joint',

We can see that there are a lot of types of places: theaters, gyms, airport infrastructure and various stores. Let's merge together Caffe and Coffe Shops in one group, and merge all restaurants to another to make clusterization more specific to types of objects (i.e. restaurant, gym, .etc) instead of slight difference among some of them (i.e. different types of restaurants)

In [68]:
toronto_venues.replace({'Venue Category': r'Café*'}, {'Venue Category': 'Coffee Shop'}, regex=True, inplace = True)
toronto_venues.replace({'Venue Category': r'.+Restaurant'}, {'Venue Category': 'Restaurant'}, regex=True, inplace = True)
toronto_venues['Venue Category'].value_counts()

Restaurant                 513
Coffee Shop                281
Park                        54
Pizza Place                 52
Bakery                      52
Sandwich Place              43
Hotel                       42
Bar                         42
Clothing Store              37
Gym                         33
Burger Joint                26
Pub                         25
Breakfast Spot              24
Gastropub                   23
Steakhouse                  22
Grocery Store               22
Diner                       21
Beer Bar                    20
Pharmacy                    20
Fried Chicken Joint         18
Bookstore                   18
Bank                        18
Furniture / Home Store      18
Gym / Fitness Center        17
Cosmetics Shop              17
Ice Cream Shop              17
Deli / Bodega               16
Dessert Shop                16
Tea Room                    16
Liquor Store                15
                          ... 
Organic Grocery              1
Recordin

After some reshaping of vanues let's create dataframe with top-10 venues in columns for each area in rows and their quantity as values:

In [74]:
#toronto_venues = toronto_venues.groupby(['Neighborhood', 'Venue Category'], as_index =False).agg({'Venue':'count'})

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Restaurant,Coffee Shop,Steakhouse,Burger Joint,Bakery,Salad Place,Bar,Pizza Place,Breakfast Spot,Cosmetics Shop
1,Agincourt,Lounge,Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Arts & Crafts Store,Yoga Studio,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pharmacy,Video Store,Fried Chicken Joint,Sandwich Place,Pizza Place,Restaurant,Beer Store,Grocery Store,Diner,Creperie
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Pharmacy,Antique Shop,College Arts Building


In [75]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [76]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = x.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged['Cluster Labels'].replace(np.nan, 4, inplace = True)

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Curling Ice,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,0.0,Restaurant,Hockey Arena,Coffee Shop,College Gym,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,2.0,Coffee Shop,Restaurant,Park,Pub,Bakery,Theater,Breakfast Spot,Spa,Beer Store,Ice Cream Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Fraternity House,Miscellaneous Shop,Carpet Store,Restaurant,Coffee Shop,Aquarium
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,0.0,Coffee Shop,Restaurant,Gym,Diner,Park,Nightclub,Music Venue,Hobby Shop,Fried Chicken Joint,Creperie


So let's plot out clusters on the map:

In [77]:
# create map
map_clusters = folium.Map(location=[43.725882,-79.315572], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters